<a href="https://colab.research.google.com/github/Jtranod98/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [817 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [85.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bioni

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
#Load Amazon Data inot Spark DataFrame
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)


In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          2|            1|          3|   N|                Y|
|          4|            0|          0|   N|                N|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          4|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          3|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|           

In [ ]:
# Retrieve all rows where total votes count is equal to or greater than 20
from pyspark.sql.functions import col,avg
tot_votes_df = vine_df.filter(col("total_votes")>=20)
tot_votes_df.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          3|           54|         59|   N|                Y|
|          5|           15|         21|   N|                Y|
|          4|           20|         21|   N|                Y|
|          2|           28|         44|   N|                Y|
|          4|           45|         47|   N|                Y|
|          4|           98|        104|   N|                Y|
|          5|           26|         39|   N|                Y|
|          5|           53|         59|   N|                Y|
|          5|           69|         77|   N|                N|
|          2|           24|         25|   N|                Y|
|          2|          103|        108|   N|                Y|
|          3|            4|         22|   N|                Y|
|          5|           57|         58|   N|           

In [ ]:
# Retrieve all rows where the number of helpful votes divided by total votes is equal to or greater than 50%
helpful_df = vine_df.filter(col("helpful_votes") /col("total_votes") >= 0.5)
helpful_df.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          1|            1|          1|   N|                Y|
|          5|            1|          1|   N|                Y|
|          5|            2|          2|   N|                Y|
|          5|           17|         18|   N|                N|
|          5|            1|          1|   N|                Y|
|          1|            1|          2|   N|                Y|
|          3|            2|          2|   N|                Y|
|          5|            1|          1|   N|                N|
|          5|            1|          1|   N|                Y|
|          5|            7|          7|   N|                Y|
|          5|            1|          1|   N|                Y|
|          5|            1|          1|   N|                Y|
|          3|           54|         59|   N|           

In [ ]:
# Retrieves all the rows where a review was written as part of the Vine program (paid)
paid_df = helpful_df.filter(col("vine") =='Y') 
paid_df.describe().show()

+-------+------------------+------------------+------------------+----+-----------------+
|summary|       star_rating|     helpful_votes|       total_votes|vine|verified_purchase|
+-------+------------------+------------------+------------------+----+-----------------+
|  count|              7598|              7598|              7598|7598|             7598|
|   mean| 3.977362463806265|10.012503290339563|11.355883127138721|null|             null|
| stddev|1.0477670361209401| 48.27650577555295| 50.78547915484564|null|             null|
|    min|                 1|                 1|                 1|   Y|                N|
|    max|                 5|              1735|              1795|   Y|                Y|
+-------+------------------+------------------+------------------+----+-----------------+



In [ ]:
# Retrieve all the rows where the review was not part of the Vine program (unpaid)
unpaid_df = helpful_df.filter(col("vine")=='N')
unpaid_df.describe().show()

+-------+------------------+-----------------+------------------+-------+-----------------+
|summary|       star_rating|    helpful_votes|       total_votes|   vine|verified_purchase|
+-------+------------------+-----------------+------------------+-------+-----------------+
|  count|           1760004|          1760004|           1760004|1760004|          1760004|
|   mean| 3.716870529839705|4.406076349826478| 5.026000509089752|   null|             null|
| stddev|1.5616846454952216|24.40815888626722|25.943987522940585|   null|             null|
|    min|                 1|                1|                 1|      N|                N|
|    max|                 5|             5854|              6063|      N|                Y|
+-------+------------------+-----------------+------------------+-------+-----------------+



In [ ]:
# Determine the total number of reviews, the number of 5-star reviews, 
# and the percentage of 5-star reviews for the review (paid).
total_paid_review = paid_df.count()
numb_paid_five_star = paid_df.filter(col("star_rating")=='5').count()
percent_paid_five_star = numb_paid_five_star / total_paid_review * 100

print("Total number of reviews: ",total_paid_review)
print("Number of 5-star reviews: ",numb_paid_five_star)
print("Percentage of 5-star reviews: ",percent_paid_five_star)


Total number of reviews:  7598
Number of 5-star reviews:  2837
Percentage of 5-star reviews:  37.3387733614109


In [ ]:
# Determine the total number of reviews, the number of 5-star reviews, 
# and the percentage of 5-star reviews for the review (unpaid).

total_unpaid_review = unpaid_df.count()
numb_unpaid_five_star = unpaid_df.filter(col("star_rating")=='5').count()
percent_unpaid_five_star = numb_unpaid_five_star / total_unpaid_review * 100

print("Total number of reviews: ",total_unpaid_review)
print("Number of 5-star reviews: ",numb_unpaid_five_star)
print("Percentage of 5-star reviews: ",percent_unpaid_five_star)

Total number of reviews:  1760004
Number of 5-star reviews:  873722
Percentage of 5-star reviews:  49.643182629130386
